In [1]:
!pip install pyspark findspark


     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 10.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0a46fa2d4c5cdde3656e24a8af57c8e1b77fd18afb1108390c044c82481d1367
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf

conf = pyspark.SparkConf().set('spark.ui.port', '4050')
sc = SparkContext.getOrCreate()

----

In [3]:
from pyspark.sql import SparkSession


ss = SparkSession(sc).builder.config("spark.driver.memory", "6g").getOrCreate()

weather_df = ss.read.option('header', True).csv("/content/drive/MyDrive/BigDataManagement/final data/weather-ta2.csv").limit(20000)
stocks_df=ss.read.option('header', True).csv('/content/drive/MyDrive/BigDataManagement/final data/stocks-ta2.csv').limit(20000)

In [4]:
weather = weather_df.rdd
stocks = stocks_df.rdd

In [5]:
weather = weather.map(lambda x: [x[1], x[2], x[3]])
stocks = stocks.map(lambda x: [x[1], x[2], x[3]])

----

In [6]:
#weather = sc.textFile('/content/drive/MyDrive/BigDataManagement/final data/weather-ta2.csv')
#header = weather.first()
#weather = weather.filter(lambda row: row != header)

In [7]:
weather.take(5)

[['01016099999', '2016-01-01', '4'],
 ['01016099999', '2016-01-02', '6'],
 ['01016099999', '2016-01-03', '0'],
 ['01016099999', '2016-01-04', '0'],
 ['01016099999', '2016-01-05', '-53']]

In [8]:
#header.split(',')[1:]

In [9]:
#weather = weather.map(lambda row: row.split(',')[1:-1])
df1 = weather.keyBy(lambda row: row[1])
df1.take(5)

[('2016-01-01', ['01016099999', '2016-01-01', '4']),
 ('2016-01-02', ['01016099999', '2016-01-02', '6']),
 ('2016-01-03', ['01016099999', '2016-01-03', '0']),
 ('2016-01-04', ['01016099999', '2016-01-04', '0']),
 ('2016-01-05', ['01016099999', '2016-01-05', '-53'])]

In [10]:
def comboid_create(row):
  el1 = row[1][0]
  el2 = row[1][1]
  comboid = [el1[0], el2[0]]
  comboid.sort()
  return ['_'.join(comboid), el1[1], el1[2], el2[2]]

df2 = df1.join(df1).filter(lambda row: row[1][0][0] > row[1][1][0]).map(comboid_create)
df2.take(5)

[['01016099999_02520099999', '2016-01-01', '6', '4'],
 ['01016099999_02741099999', '2016-01-01', '22', '4'],
 ['02520099999_02741099999', '2016-01-01', '22', '6'],
 ['01016099999_02845099999', '2016-01-01', '-32', '4'],
 ['02520099999_02845099999', '2016-01-01', '-32', '6']]

In [11]:
def max(a,b):
  if float(a) > float(b):
    return float(a)
  else:
    return float(b)

def min(a,b):
  if float(a) > float(b):
    return float(b)
  else:
    return float(a)

def avg(a,b):
  return (float(a) + float(b))/2

def aggregate(row):
  a = row[-2]
  b = row[-1]
  return [row[0], row[1], min(a,b), avg(a,b), max(a,b)]

df3 = df2.map(aggregate)
df3.take(5)

[['01016099999_02520099999', '2016-01-01', 4.0, 5.0, 6.0],
 ['01016099999_02741099999', '2016-01-01', 4.0, 13.0, 22.0],
 ['02520099999_02741099999', '2016-01-01', 6.0, 14.0, 22.0],
 ['01016099999_02845099999', '2016-01-01', -32.0, -14.0, 4.0],
 ['02520099999_02845099999', '2016-01-01', -32.0, -13.0, 6.0]]

combo_id, date, min, avg, max

-----

In [12]:
stocks.take(5)

[['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-01',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-02',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-03',
  '57.5'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-04',
  '57.8'],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  '2016-01-05',
  '57.17']]

In [13]:
stocks = stocks.map(lambda row: [row[0], row[1], float(row[2])]).keyBy(lambda row: row[1])

-----

In [14]:
bigdata = stocks.join(df3.keyBy(lambda row: row[1]))
bigdata.take(1)

[('2016-01-02',
  (['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
    '2016-01-02',
    57.5],
   ['01016099999_02520099999', '2016-01-02', 4.0, 5.0, 6.0]))]

In [15]:
def parse_after_join(row):
  el1 = row[1][0]
  el2 = row[1][1]
  return [el1[0], el2[0], [el1[-1]], [el2[-3]], [el2[-2]], [el2[-1]]]

bigdata = bigdata.map(parse_after_join)
bigdata.first()

['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
 '01016099999_02520099999',
 [57.5],
 [4.0],
 [5.0],
 [6.0]]

(stock_id, combo_id, price, min, avg, max)

In [17]:
agg = bigdata.keyBy(lambda row: (row[0], row[1])).reduceByKey(lambda a,b: [a[-4] + b[-4], a[-3] + b[-3],a[-2] + b[-2],a[-1] + b[-1]])

In [19]:
import numpy as np

def cosine(a, b):
  return float(np.dot(np.array(a), np.array(b)) / (np.linalg.norm(np.array(a)) * np.linalg.norm(np.array(b))))

In [30]:
def parser(key):
  return key[:-12], key[-11:]

In [35]:
def get_cosine(tuple_row):
  price_min = cosine(tuple_row[1][0],tuple_row[1][1])
  price_avg = cosine(tuple_row[1][0],tuple_row[1][2])
  price_max = cosine(tuple_row[1][0],tuple_row[1][3])
  stock, combo_id = tuple_row[0]
  station1, station2 = parser(combo_id)
  return [stock, station1, station2, price_min, price_avg, price_max]

In [36]:
results = agg.map(get_cosine)

(stock_name, station_id_1, station_id_2, price_min_cossim, price_avg_cossim, price_max_cossim)

In [39]:
results.sortBy(lambda row: row[-2], ascending=False).take(10)

[['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '06207099999',
  '08524099999',
  0.9046139361446396,
  0.9680027404971031,
  0.9889081391030721],
 ['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '03066099999',
  '08524099999',
  0.8502761184041757,
  0.9667747023429623,
  0.989104791777618],
 ['10698.Asien--Australien--Südamerika--Afrika_Japan_Tiemco-Ltd._0493AO',
  '06207099999',
  '08524099999',
  0.9026194377133125,
  0.964841150386873,
  0.985040564938908],
 ['10612.Asien--Australien--Südamerika--Afrika_Japan_Techno-Alpha_0492WS',
  '06207099999',
  '08524099999',
  0.9006077362167587,
  0.964636354070597,
  0.9860404654101109],
 ['10679.Asien--Australien--Südamerika--Afrika_Japan_The-Michinoku-Bank_0490JR',
  '03862099999',
  '08524099999',
  0.8536509536957989,
  0.9641612314038129,
  0.9890492205640301],
 ['1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404',
  